In [202]:
import pandas as pd
import re
import helpers as hpr

In [203]:
df = pd.DataFrame([])
data_path = "Data/"
changes_file_names = hpr.list_file(data_path)
for i in range(len(changes_file_names)):
    # df_per_file = pd.read_csv("%s/%s" % (data_path, f))
    df_per_file = pd.read_csv("%schanges_data_%d.csv" % (data_path, i))
    df = pd.concat((df, df_per_file))

In [204]:
df = df.reset_index(drop=True).sort_values(by="updated", ascending=False)

In [205]:
len(df)

703048

In [206]:
df[["id"]].isnull().count() # No emty id

id    703048
dtype: int64

In [225]:
def retrieve_depends_on(x):
    result = re.split(r"\n|\r", x)

    final_result = []
    for l in result:
        if re.search(r"Depends-On:\s[a-zA-Z0-9]+", l):
            if l.startswith("Depends-On: https://review"):
                final_result.append(str(re.search(r"\d+", l)[0]))
            elif re.search(r"^Depends-On:\s[^@:%._\\+~#?&//=]", l):
                if l.find("https") != -1:
                    continue

                final_result.append(l[12:])
    return final_result if len(final_result) != 0 else None


def retrieve_closes_bug(x):
    result = re.search(r"(Closes-Bug:\s#\d+)", x)
    return result[0][13:] if result else None

In [235]:
df_subset = df.head(1000).copy()

In [236]:
df_subset["depends_on"] = df_subset["commit_message"].map(retrieve_depends_on)

In [237]:
df_subset = df_subset[df_subset["depends_on"].isnull() == False].reset_index(drop=True)

In [238]:
df_subset.head(5)

,id,project,branch,topic,change_id,subject,status,created,updated,submitted,...,current_revision,discussion_messages_count,reviewers_count,revisions_count,files_count,owner_account_id,owner_name,owner_username,commit_message,depends_on
0,openstack%2Fdevstack-plugin-ceph~master~I89982...,openstack/devstack-plugin-ceph,master,unpin-focal,I899822fec863f43cd6c58b25cf4688c6a3ac1e9b,"Revert ""Temporary pin the ceph jobs nodeset to...",NEW,2022-11-22 17:32:50.000000000,2022-11-22 21:13:04.000000000,NaN,...,5941bb9df5b87b1d98da5424465c2f2b29fd34eb,6,1,2,1,13252,Dr. Jens Harbott,jrosenboom,"Revert ""Temporary pin the ceph jobs nodeset to...",[865314]
1,openstack%2Fnova~stable%2Fwallaby~I4b400d499b9...,openstack/nova,stable/wallaby,bug/1970467,I4b400d499b9f594798f2da6d16f21204371a9f42,Detect port-resource-request-groups neutron AP...,NEW,2022-11-22 20:59:56.000000000,2022-11-22 20:59:56.000000000,NaN,...,eaf70f06a136c86a20d97191b7f801038c81f751,1,0,1,3,11604,sean mooney,sean-k-mooney,Detect port-resource-request-groups neutron AP...,[803905]
2,openstack%2Ftripleo-heat-templates~master~If4e...,openstack/tripleo-heat-templates,master,NaN,If4e74edb03037fa156dbc765e1b665d4b9b5b192,Allow to skip container-image-prepare task,NEW,2022-10-06 05:39:19.000000000,2022-11-22 20:45:23.000000000,NaN,...,bbba6157476347f5f0eaf9f88b117e6ade6651fe,5,4,1,1,8833,Rabi Mishra,rabi,Allow to skip container-image-prepare task\r\n...,[860524]
3,openstack%2Fneutron~master~I8164d76b28904cde88...,openstack/neutron,master,improve-neutron-ci,I8164d76b28904cde88f4de7da08d647f6bd98a10,Add tempest job which will run on the oldest U...,NEW,2022-10-24 13:07:58.000000000,2022-11-22 20:39:07.000000000,NaN,...,01b11df1b5652fbd11f941c008892261714e46a3,40,6,3,3,11975,Slawek Kaplonski,slaweq,Add tempest job which will run on the oldest U...,"[861110, 862219, 862479]"
4,openstack%2Ftripleo-ansible~stable%2Ftrain~I61...,openstack/tripleo-ansible,stable/train,ceph_ansible_log_rotate,I61f80b11c9394f34e09297ca1aa93d7000bb8047,[TRAIN-ONLY] Rotate ceph_ansible_command.log,NEW,2022-11-21 23:16:36.000000000,2022-11-22 20:38:22.000000000,NaN,...,22e9588f9d89f385e5fbbe7deb435de0fbafae15,9,6,1,1,18002,John Fulton,fultonj,[TRAIN-ONLY] Rotate ceph_ansible_command.log\r...,[Ifbd990f15b9d85260ab20a2004fe79bafda929d9]


In [243]:
df_subset = df_subset.explode(column="depends_on").reset_index(drop=True)

In [244]:
df_subset.loc[df_subset["change_id"] == "I8164d76b28904cde88f4de7da08d647f6bd98a10", "depends_on"]

3    861110
4    862219
5    862479
Name: depends_on, dtype: object

In [245]:
df["depends_on"] = df["commit_message"].map(retrieve_depends_on)

In [247]:
df = df.explode(column="depends_on").reset_index(drop=True)

In [248]:
df[["id"]].describe()

,id
count,707230
unique,702912
top,openstack%2Ftripleo-ci~master~I2c0726b2234e0b7...
freq,44


In [249]:
len(df)

707230

In [251]:
df[["depends_on"]].describe()

,depends_on
count,31690
unique,22197
top,555034
freq,222


In [252]:
df[["depends_on"]].value_counts().to_csv("./experiments/depends_on.csv")

In [254]:
# df[["depends_on"]].value_counts().to_csv("./experiments/depends_on.csv")
df[df["depends_on"].isnull() == False].sort_values(
    by="depends_on", ascending=False, ignore_index=True)[[
        "id", "project", "branch", "topic", "change_id", "subject", "status",
        "created", "updated", "submitted", "insertions", "deletions",
        "total_comment_count", "number", "discussion_messages_count",
        "reviewers_count", "revisions_count", "files_count", "depends_on"
    ]].to_csv("./experiments/depends_on_all.csv", index=False)


In [257]:
def build_depends_chain(row):
    obj = {}
    depends_on = row["depends_on"]
    obj["Target"] = row["change_id"]
    obj["Target_repo"] = row["project"]
    row_src = None
    if depends_on.isnumeric():
        row_src = df[df["number"] == int(depends_on)].head(1)
    else:
        row_src = df[df["change_id"] == depends_on].head(1)

    if len(row_src) != 0:
        obj["Source"] = row_src["change_id"].tolist()[0]
        obj["Source_repo"] = row_src["project"].tolist()[0]

    return obj

In [258]:
df_subset_columns = ["change_id", "project", "depends_on", "number"]

df_subset_dep = df.loc[df["depends_on"].isnull() == False,
                       df_subset_columns].copy().reset_index(drop=True)

df_depends_on = df_subset_dep.apply(build_depends_chain, axis=1)

df_depends_on = pd.json_normalize(data=df_depends_on, errors="ignore")

In [262]:
evolution_columns = ["Source", "Target", "Source_repo", "Target_repo"]

df_depends_on = df_depends_on.loc[:, evolution_columns]

In [ ]:
df_depends_on.to_csv("openstack_evolution.csv", index=False)

In [ ]:
df_depends_on[["Source", "Target", "Source_repo", "Target_repo"
               ]].value_counts().to_csv("./experiments/duplicates.csv")

In [285]:
df_depends_on = pd.read_csv("./openstack_evolution.csv")

In [287]:
df_depends_on.count()

Source         30463
Target         31690
Source_repo    30463
Target_repo    31690
dtype: int64

In [286]:
df_depends_on.isnull().sum()

Source         1227
Target            0
Source_repo    1227
Target_repo       0
dtype: int64

In [288]:
df_depends_on = df_depends_on.dropna()

In [290]:
df_depends_on.count()

Source         30463
Target         30463
Source_repo    30463
Target_repo    30463
dtype: int64

In [289]:
df_depends_on.isnull().sum()

Source         0
Target         0
Source_repo    0
Target_repo    0
dtype: int64

In [292]:
df_depends_on.duplicated().sum()

3070

In [293]:
df_depends_on = df_depends_on.drop_duplicates()

In [294]:
df_depends_on.duplicated().sum()

0

In [298]:
df_depends_on[["Source_repo", "Target_repo"]].duplicated().sum()

22051

In [299]:
df_depends_on = df_depends_on.drop_duplicates(subset=["Source_repo", "Target_repo"])

In [300]:
df_depends_on[["Source_repo", "Target_repo"]].duplicated().sum()

0

In [301]:
df_depends_on.count()

Source         5342
Target         5342
Source_repo    5342
Target_repo    5342
dtype: int64

In [302]:
df_depends_on = df_depends_on.reset_index(drop=True)

In [304]:
df_depends_on.to_csv("clean_openstack_evolution.csv", index=False)